In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from datasets import load_dataset, concatenate_datasets, Dataset
import copy
from tqdm import tqdm  # For tracking training progress


In [34]:
import torch
import numpy as np
import random

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [35]:
# Example list of labels
label_names = ['cat', 'dog', 'bird', 'fish', 'car', 'aircraft', 'flower', 'truck', 'parachute', 'mushroom']

# Create a mapping from label names to indices
label_to_index = {label: idx for idx, label in enumerate(label_names)}


In [36]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3), # Convert grayscale to 3 channels (RGB)
    transforms.Resize((256, 256)),  # Resize all images to 256x256
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])


In [37]:
from torch.utils.data import Dataset, random_split

class CustomImageDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
        self.label_to_index = {label: idx for idx, label in enumerate(label_names)}

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        image = sample['image']
        label=sample['label']
        label_index = self.label_to_index[label]
        if self.transform:
            image = self.transform(image)
        label_tensor = torch.tensor(label_index, dtype=torch.long)
        return image, label_tensor # Replace 'label' with actual label field if available

def prepare_custom_dataloader(client_combined, batch_size=16, split_ratio=0.2, seed=42):
    
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    custom_dataset = CustomImageDataset(client_combined, transform=transform)
    print("Custom dataset size:", len(custom_dataset))
    # Calculate sizes for train/test splits
    test_size = int(split_ratio * len(custom_dataset))
    train_size = len(custom_dataset) - test_size

    # Split the dataset into train and test sets
    train_dataset, test_dataset = random_split(custom_dataset, [train_size, test_size], generator=torch.Generator().manual_seed(seed))
    # Create DataLoaders for train and test sets
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    print("Train loader size:", len(train_loader))

    return train_loader, test_loader


In [38]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        # Define layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 64 * 64, 128)  # Adjust based on output size from conv layers
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))  # Conv Layer 1
        x = self.pool((nn.ReLU()(self.conv2(x))))  # Conv Layer 2
        x = x.view(-1, 32 * 64 * 64)  # Flatten for fully connected layer
        x = nn.ReLU()(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x) 
        return x

In [39]:
# Function to update local model on each client
def train_local_model(model, train_loader, epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.train()
    
    for epoch in range(epochs):
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    return model.state_dict()  # Return the trained model's weights

# Function to average weights across clients
def federated_avg(global_model, client_weights):
    avg_weights = copy.deepcopy(client_weights[0])
    
    for key in avg_weights.keys():
        for i in range(1, len(client_weights)):
            avg_weights[key] += client_weights[i][key]
        avg_weights[key] = avg_weights[key] / len(client_weights)
    
    global_model.load_state_dict(avg_weights)
    return global_model


In [40]:
client1_dataset = load_dataset('AnnantJain/client1_federated_dataset')
client2_dataset = load_dataset('AnnantJain/client2_federated_dataset')
client3_dataset = load_dataset('AnnantJain/client3_federated_dataset')
client4_dataset = load_dataset('AnnantJain/client4_federated_dataset')
client5_dataset = load_dataset('AnnantJain/client5_federated_dataset')

Using the latest cached version of the dataset since AnnantJain/client1_federated_dataset couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\annan\.cache\huggingface\datasets\AnnantJain___client1_federated_dataset\default\0.0.0\7b1dada41914df35469672595f2dd09230fdd9b9 (last modified on Mon Sep 30 15:22:06 2024).


In [41]:
# Combine clean and noisy data for each client
client1_combined = concatenate_datasets([client1_dataset['clean'], client1_dataset['noisy']])
client2_combined = concatenate_datasets([client2_dataset['clean'], client2_dataset['noisy']])
client3_combined = concatenate_datasets([client3_dataset['clean'], client3_dataset['noisy']])
client4_combined = concatenate_datasets([client4_dataset['clean'], client4_dataset['noisy']])
client5_combined = concatenate_datasets([client5_dataset['clean'], client5_dataset['noisy']])


In [42]:
train_loader_1, test_loader_1 = prepare_custom_dataloader(client1_combined)
train_loader_2, test_loader_2 = prepare_custom_dataloader(client2_combined)
train_loader_3, test_loader_3 = prepare_custom_dataloader(client3_combined)
train_loader_4, test_loader_4 = prepare_custom_dataloader(client4_combined)
train_loader_5, test_loader_5 = prepare_custom_dataloader(client5_combined)

Custom dataset size: 3000
Train loader size: 150
Custom dataset size: 2296
Train loader size: 115
Custom dataset size: 2000
Train loader size: 100
Custom dataset size: 1800
Train loader size: 90
Custom dataset size: 3200
Train loader size: 160


In [43]:
for images, labels in train_loader_1:
    # Since DataLoader returns batches, you may want to just take the first batch
    break  # Exit after the first batch

# Now 'images' and 'labels' are tensors
print("Sample Image Tensor Shape:", images.shape)  # Print the shape of the image tensor
print("Sample Label Tensor:", labels)  # Print the label tensor

Sample Image Tensor Shape: torch.Size([16, 3, 256, 256])
Sample Label Tensor: tensor([8, 5, 2, 2, 8, 8, 5, 5, 5, 2, 8, 8, 5, 5, 8, 5])


In [44]:
# Initialize global model (shared across clients)
num_classes = 10  # Adjust based on your specific case
global_model = SimpleCNN(num_classes=num_classes)


In [45]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [46]:
# Number of communication rounds
num_rounds = 5

# Number of local epochs for each client
local_epochs = 2

# Perform Federated Averaging
for round_num in range(num_rounds):
    print(f"Round {round_num + 1}/{num_rounds}")
    
    # Collect weights from all clients
    client_weights = []
    
    # Simulate client training
    for client_id, train_loader in enumerate([train_loader_1, train_loader_2, train_loader_3, train_loader_4, train_loader_5]):
        local_model = copy.deepcopy(global_model)  # Each client starts from the global model
        local_weights = train_local_model(local_model, train_loader, epochs=local_epochs)  # Train locally
        client_weights.append(local_weights)  # Store client weights
    
    # Federated averaging (aggregation) step
    global_model = federated_avg(global_model, client_weights)
    
    print(f"Completed round {round_num + 1}")

    # Evaluate the global model on the test data from each client
    for client_id, test_loader in enumerate([test_loader_1, test_loader_2, test_loader_3, test_loader_4, test_loader_5]):
        accuracy = evaluate_model(global_model, test_loader)
        print(f"Client {client_id + 1} Test Accuracy: {accuracy:.2f}%")


Round 1/5
Completed round 1
Client 1 Test Accuracy: 16.50%
Client 2 Test Accuracy: 30.72%
Client 3 Test Accuracy: 0.00%
Client 4 Test Accuracy: 18.33%
Client 5 Test Accuracy: 13.75%
Round 2/5
Completed round 2
Client 1 Test Accuracy: 22.17%
Client 2 Test Accuracy: 8.50%
Client 3 Test Accuracy: 11.75%
Client 4 Test Accuracy: 62.78%
Client 5 Test Accuracy: 43.44%
Round 3/5
Completed round 3
Client 1 Test Accuracy: 43.33%
Client 2 Test Accuracy: 23.97%
Client 3 Test Accuracy: 7.50%
Client 4 Test Accuracy: 57.50%
Client 5 Test Accuracy: 49.06%
Round 4/5
Completed round 4
Client 1 Test Accuracy: 56.33%
Client 2 Test Accuracy: 21.57%
Client 3 Test Accuracy: 9.75%
Client 4 Test Accuracy: 64.17%
Client 5 Test Accuracy: 54.69%
Round 5/5
Completed round 5
Client 1 Test Accuracy: 59.00%
Client 2 Test Accuracy: 38.78%
Client 3 Test Accuracy: 17.25%
Client 4 Test Accuracy: 70.28%
Client 5 Test Accuracy: 42.03%
